https://deap.readthedocs.io/en/master/examples/ga_onemax_numpy.html

- binary list
- eva max

In [10]:
import random

from deap import base
from deap import creator
from deap import tools

In [11]:
import array
import numpy

from deap import algorithms

In [12]:
# IND_SIZE = 1
# NGEN = 1000
# FREQ = 100
# CXPB, MUTPB = 0.5, 0.2

In [13]:
import numpy
a = numpy.array((1,2,3,4))
b = numpy.array((5,6,7,8))
a[1:3], b[1:3] = b[1:3], a[1:3]
print(a)
print(b)

# copyしたものをswapしないといけない

[1 6 7 4]
[5 6 7 8]


In [14]:
import numpy
a = numpy.array((1,2,3,4))
b = numpy.array((5,6,7,8))
a[1:3], b[1:3] = b[1:3].copy(), a[1:3].copy()
print(a)
print(b)

[1 6 7 4]
[5 2 3 8]


In [15]:
# 普通のone maxの例と同じ。やはりalgorithms使う場合も、ここは変わらないわけだ。

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
#creator.create("Individual", list, fitness=creator.FitnessMax)
#creator.create("Individual", array.array, typecode='b', fitness=creator.FitnessMax)
creator.create("Individual", numpy.ndarray, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalOneMax(individual):
    return sum(individual),


def cxTwoPointCopy(ind1, ind2):
    """Execute a two points crossover with copy on the input individuals. The
    copy is required because the slicing in numpy returns a view of the data,
    which leads to a self overwritting in the swap operation. It prevents
    ::
    
        >>> import numpy
        >>> a = numpy.array((1,2,3,4))
        >>> b = numpy.array((5.6.7.8))
        >>> a[1:3], b[1:3] = b[1:3], a[1:3]
        >>> print(a)
        [1 6 7 4]
        >>> print(b)
        [5 6 7 8]
    """
    size = len(ind1)
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] \
        = ind2[cxpoint1:cxpoint2].copy(), ind1[cxpoint1:cxpoint2].copy()
        
    return ind1, ind2
    
    

toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", cxTwoPointCopy)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [16]:
# 違うのはmainの中のalgorithmsを使うところ。アルゴリズムを自前で作らなくていいところ。

def main():
    random.seed(64)
        
    pop = toolbox.population(n=300)
    #hof = tools.HallOfFame(1)
    hof = tools.HallOfFame(1, similar=numpy.array_equal)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, 
                                   stats=stats, halloffame=hof, verbose=True)
    
    return pop, log, hof



    '''
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    fits = [ind.fitness.values[0] for ind in pop]
    g = 0
    
    # Begin the evolution
    while max(fits) < 100 and g < 1000:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))
        
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        # 交叉や突然変異の対象になり、評価値を消したものがinvalidになっている。
        # 非対象の個体はvalidで評価値がある状態だから、評価値計算は不要
        # だからinvalidだけ評価値を計算している
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
            
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std) 
    '''


In [17]:
main()

gen	nevals	avg    	std    	min	max
0  	300   	50.4933	5.33822	34 	64 
1  	181   	54.8333	4.34958	44 	66 
2  	191   	58.4567	3.45564	47 	68 
3  	199   	60.9533	2.9025 	52 	68 
4  	167   	62.96  	2.90719	47 	71 
5  	175   	64.99  	2.84896	57 	73 
6  	168   	66.9333	2.80515	58 	74 
7  	187   	68.9167	2.82661	59 	76 
8  	171   	70.8867	2.4455 	62 	76 
9  	155   	72.69  	2.62435	62 	80 
10 	171   	74.1233	2.61052	64 	82 
11 	191   	75.64  	2.70007	65 	82 
12 	171   	77.18  	2.55752	69 	84 
13 	173   	78.7667	2.24475	69 	84 
14 	185   	79.9067	2.36459	72 	86 
15 	205   	81.4433	2.3805 	72 	88 
16 	163   	82.6767	2.22534	74 	88 
17 	175   	83.6833	2.37411	76 	88 
18 	181   	84.8067	2.30274	74 	90 
19 	179   	85.6233	2.51955	74 	91 
20 	178   	86.58  	2.16416	78 	91 
21 	173   	87.2533	2.33148	78 	91 
22 	155   	88.06  	2.15787	79 	92 
23 	187   	88.37  	2.20146	80 	92 
24 	184   	89.2767	1.97825	82 	94 
25 	198   	89.7767	2.3805 	80 	95 
26 	185   	90.6233	2.41553	80 	96 
27 	160   	91.62  	2

([Individual([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  Individual([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  Individual([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1,